Grupo 3 - miniproyecto Deep Learning

# Datos y librerias

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [42]:
# Importación librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense

In [3]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('trainReg.csv')
dataTesting = pd.read_csv('testReg.csv')

In [62]:
dataTraining = dataTraining.set_index('ID')
dataTraining.head()

,Y,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90
ID,,,,,,,,,,,,,,,,,,,,,
1,2013,44.81144,0.83826,0,7.91314,10.94148,-0.04547,-15.16332,-10.47324,14.17212,...,-8.44873,-230.30484,-40.94698,48.20025,-0.28694,155.76251,-56.23579,13.62599,123.92018,10.02845
2,1998,41.99180,7.99976,0,64.26707,16.54115,-9.28737,-40.73524,33.60440,9.18802,...,18.68972,-44.06062,52.37792,81.36093,-14.81111,151.66273,-120.61213,10.57519,-3.21078,-1.07438
3,1998,42.19196,2.23111,0,65.07719,24.99746,1.76100,6.66573,3.45778,-24.42779,...,-3.69878,-118.95712,54.15529,-23.32168,-9.65067,-83.83055,-141.17594,7.33084,-275.69714,2.35522
4,1998,39.28634,-1.85716,0,91.04190,9.08333,0.08502,-5.59216,65.62463,8.33105,...,20.89044,-18.53135,176.09769,351.33669,3.44682,121.69156,-270.43989,12.51659,-140.88884,-0.23476
5,1998,40.36025,2.94918,0,53.83723,13.71369,-8.21964,-40.21636,21.22366,17.16742,...,19.91979,34.59026,-69.83720,102.31946,8.08807,135.08089,-153.02327,4.09207,-68.33046,-6.19159


In [60]:
dataTesting = dataTesting.set_index('ID')
dataTesting.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90
ID,,,,,,,,,,,,,,,,,,,,,
5972,35.74170,0.86615,0,1.44450,-13.87834,-9.57602,5.89475,-4.46036,-10.60501,-2.20779,...,10.78210,-183.63792,131.18868,37.68626,-4.48926,10.98139,-48.14549,-7.27992,-54.52826,4.81414
57116,35.31664,-0.47799,0,-12.27958,-29.32847,-2.38610,-7.27386,-1.87712,-3.74953,-3.38237,...,53.92478,-238.77713,134.28149,81.19403,-4.81669,106.00178,3.34940,2.72328,156.94894,-8.44873
29076,43.25032,-6.31920,0,-2.56469,-7.59368,2.12862,23.29535,-0.34225,4.48761,4.93676,...,40.95711,-269.59803,69.88747,29.65701,-0.12854,106.92919,-62.86550,30.12278,24.39843,-10.68316
83358,44.94220,-4.12303,0,-15.79205,7.15794,1.46007,-3.38280,-8.48270,12.69732,-1.68102,...,0.81574,-243.09497,24.20334,-10.02719,19.25667,66.15602,-64.78646,9.09148,11.46055,-7.12136
84664,50.40813,-1.52061,0,-10.10142,-17.74876,-5.96768,-29.94136,-11.84237,5.90144,3.95996,...,-3.58634,-61.67109,47.53814,139.04916,6.85904,-57.09356,-49.57683,-6.61453,22.55497,-4.39272


In [77]:
def guardar(ypred,nombre):
    ypred = pd.DataFrame(ypred,index=dataTesting.index, columns=['Y'])
    # Guardar predicciones en formato exigido en la competencia de kaggle
    archivo = nombre + '.csv'
    ypred.to_csv(archivo, index_label='ID')
    return ypred

In [26]:
def calculo_rmse(ypred,yTest,ypredTr,yTrain):
    RMSE = round(mse(ypred, yTest, squared = False))
    MSE = round(mse(ypred, yTest))
    #MAE = round(mae(ypred, yTest))

    RMSE_Tr = round(mse(ypredTr, yTrain, squared = False))
    MSE_Tr = round(mse(ypredTr, yTrain))
    #MAE_Tr = round(mae(ypredTr, yTrain))

    # Cálculo del desempeño del modelo utilizando el RMSE y el MAE
    print(f'El RMSE del modelo es de {RMSE}')
    print(f'El MSE del modelo es de {MSE}')
    #print(f'El MAE del modelo es de {MAE}')

    print(f'El RMSE del modelo en train es de {RMSE_Tr}')
    print(f'El MSE del modelo en train es de {MSE_Tr}')
    #print(f'El MAE del modelo en train es de {MAE_Tr}')

# Exploración de datos

In [63]:
dTr = dataTraining.copy()

In [11]:
dTr.describe(include="all").T

,count,mean,std,min,25%,50%,75%,max
ID,77779.0,48244.238342,27846.283673,1.00000,24079.500000,48357.00000,72340.500000,96435.00000
Y,77779.0,2002.308129,10.811038,1926.00000,1998.000000,2006.00000,2010.000000,2014.00000
V1,77779.0,43.425185,6.128869,4.83688,40.060315,44.32385,47.900080,60.03401
V2,77779.0,-0.136720,4.370466,-69.68087,-2.612435,-0.06300,2.465950,23.81526
V3,77779.0,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.00000
...,...,...,...,...,...,...,...,...
V86,77779.0,17.575097,115.243644,-3168.92457,-31.569390,15.23736,67.361475,2144.10391
V87,77779.0,-25.628857,173.310304,-4319.99232,-100.668480,-21.58164,51.333150,2833.60895
V88,77779.0,4.463484,13.526414,-236.03926,-2.569360,3.13690,10.002145,275.35366
V89,77779.0,18.664885,186.690183,-7458.37815,-60.121980,5.94149,84.389125,5289.11138


In [14]:
sum(dTr.isnull().sum() / dTr.shape[0] * 100)

0.0

# Preparación de datos

In [66]:
XTotal = dTr.loc[:,dTr.columns!='Y']
yTotal = dTr.loc[:,dTr.columns=='Y']

In [67]:
XTrain, XTest, yTrain, yTest = train_test_split(XTotal, yTotal, test_size=0.30, random_state=3)

In [68]:
scaler = StandardScaler(with_mean=True, with_std=True)
scaler.fit(XTrain)
XTrainS=scaler.transform(XTrain)
XTestS=scaler.transform(XTest)
dTestS=scaler.transform(dataTesting)

In [80]:
yTrain.to_numpy()

array([[1977],
       [1944],
       [2005],
       ...,
       [2010],
       [2008],
       [2011]], dtype=int64)

# Análisis preliminar de modelos

## Keras

In [84]:
# Definición del modelo
model = Sequential()
model.add(Dense(64, input_dim=90, activation='relu'))  # Capa oculta 1 con 64 neuronas
model.add(Dense(64, activation='relu'))               # Capa oculta 2 con 64 neuronas
model.add(Dense(64, activation='relu'))               # Capa oculta 3 con 64 neuronas
model.add(Dense(1))                                   # Capa de salida con 1 neurona (regresión)

# Compilación del modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# Entrenamiento del modelo
model.fit(XTrainS, yTrain.to_numpy(), epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50


AttributeError: module 'keras.src.backend' has no attribute 'convert_to_numpy'

## Scikit Learn

In [72]:
model = MLPRegressor(hidden_layer_sizes=(128, 128, 128), activation='relu', solver='lbfgs', 
                     max_iter=500, random_state=42)
model.fit(XTrainS, yTrain)

MLPRegressor(hidden_layer_sizes=(128, 128, 128), max_iter=500, random_state=42,
             solver='lbfgs')

In [73]:
predNN=model.predict(XTestS)
predNNTr=model.predict(XTrainS)

In [74]:
calculo_rmse(predNN,yTest,predNNTr,yTrain)

El RMSE del modelo es de 13
El MSE del modelo es de 165
El RMSE del modelo en train es de 9
El MSE del modelo en train es de 79


# Entrenamiento y calibración

In [78]:
predNN1=model.predict(dTestS)
guardar(predNN1,'Skl_NN4')

,Y
ID,
5972,2005.826770
57116,1992.707868
29076,1999.214673
83358,2001.212872
84664,2005.111924
...,...
5763,2011.702906
25441,2000.864693
53318,1997.623429


# Visualización de resultados